In [1]:
from coffea import processor
from coffea.analysis_tools import PackedSelection, Cutflow
import awkward as ak
import pandas as pd
import dask_awkward as dak
import hist.dask as hda
from collections import namedtuple
import hist
import vector
vector.register_awkward()

In [2]:

##########################
# Define plot properties #
##########################
plot_props = pd.DataFrame({
    'Zm':{'name':'Zm','title':'Z Candidate mass','xlabel':'$Z_{mass}$ [GeV]','ylabel':'Events','bins':100,'xmin':0,'xmax':250},
    'Zm_zoom':{'name':'Zm_zoom','title':'Z Candidate mass','xlabel':'$Z_{mass}$ [GeV]','ylabel':'Events','bins':40,'xmin':80,'xmax':100},
    'Recoilm':{'name':'Recoilm','title':'Leptonic Recoil mass','xlabel':'$Recoil_{mass}$ [GeV]','ylabel':'Events','bins':100,'xmin':0,'xmax':200},
    'Recoilm_zoom':{'name':'Recoilm_zoom','title':'Leptonic Recoil mass','xlabel':'$Recoil_{mass}$ [GeV]','ylabel':'Events','bins':200,'xmin':80,'xmax':160},
    'Recoilm_zoom1':{'name':'Recoilm_zoom1','title':'Leptonic Recoil mass','xlabel':'$Recoil_{mass}$ [GeV]','ylabel':'Events','bins':100,'xmin':120,'xmax':140},
    'Recoilm_zoom2':{'name':'Recoilm_zoom2','title':'Leptonic Recoil mass','xlabel':'$Recoil_{mass}$ [GeV]','ylabel':'Events','bins':200,'xmin':120,'xmax':140},
    'Recoilm_zoom3':{'name':'Recoilm_zoom3','title':'Leptonic Recoil mass','xlabel':'$Recoil_{mass}$ [GeV]','ylabel':'Events','bins':400,'xmin':120,'xmax':140},
    'Recoilm_zoom4':{'name':'Recoilm_zoom4','title':'Leptonic Recoil mass','xlabel':'$Recoil_{mass}$ [GeV]','ylabel':'Events','bins':800,'xmin':120,'xmax':140},
    'Recoilm_zoom5':{'name':'Recoilm_zoom5','title':'Leptonic Recoil mass','xlabel':'$Recoil_{mass}$ [GeV]','ylabel':'Events','bins':2000,'xmin':120,'xmax':140},
    'Recoilm_zoom6':{'name':'Recoilm_zoom6','title':'Leptonic Recoil mass','xlabel':'$Recoil_{mass}$ [GeV]','ylabel':'Events','bins':100,'xmin':130.3,'xmax':140}
})

def get_1Dhist(name, var, flatten=False):
    '''
    name: eg. Zm
    var: eg. variable containing array of mass of Z
    flatten: If to flatten var before fill; True by default
    Returns a histogram
    '''
    props = plot_props[name]
    if flatten : var = dak.ravel(var) # Removes None values and all the nesting
    var = var[~dak.is_none(var, axis=0)] # Remove None values only
    return hda.Hist.new.Reg(props.bins, props.xmin, props.xmax).Double().fill(var)

def get(events,collection,attribute,*cut):
    '''
    Get an attribute from a branch with or without a base cut.
    '''
    if len(cut) != 0:
        return events[collection+'/'+collection+'.'+attribute][cut[0]]
    return events[collection+'/'+collection+'.'+attribute]

def get_all(events,Collection,*basecut):
    '''
    Collect all the attributes of a collection into a namedtuple named particle, with or without a base cut
    '''
    prefix = '/'.join([Collection]*2)+'.'
    list_of_attr = [field.replace(prefix,'') for field in events.fields if field.startswith(prefix)]
    replace_list = ['.','[',']']
    valid_attr = list_of_attr
    for rep in replace_list:
        valid_attr = [field.replace(rep, '_') for field in valid_attr ]
    part = namedtuple('particle', valid_attr)
    return part(*[get(events,Collection,attr,*basecut) for attr in list_of_attr])

def get_reco(Reconstr_branch, needed_particle, events):
    '''
    Match the Reconstructed collection to the desired particle collection.
    '''
    part = namedtuple('particle', list(Reconstr_branch._fields))
    return part(*[getattr(Reconstr_branch,attr)[get(events,needed_particle,'index')] for attr in Reconstr_branch._fields])

def Reso_builder(lepton, resonance):
    '''
    Builds Resonance candidates
    Input:    lepton(var*[var*LorentzVector]),
              resonance(float)
    Output: Reso([var*LorentzVecctor]) best resonance candidate in each event (maximum one per event)
    '''
    #Create all the combinations
    combs = dak.combinations(lepton,2)
    # Get dileptons
    lep1 , lep2 = dak.unzip(combs)
    di_lep = lep1 + lep2 # This process drops any other field except 4 momentum fields

    di_lep = ak.zip({"px":di_lep.px,"py":di_lep.py,"pz":di_lep.pz,"E":di_lep.E,"q":lep1.q + lep2.q,}, with_name="Momentum4D")
    
    # Sort by closest mass to the resonance value
    sort_mask = dak.argsort(abs(resonance-abs(di_lep.mass)), axis=1)
    Reso = di_lep[sort_mask]

    #Choose the best candidate
    Reso = dak.fill_none(Reso,[],axis=0) #Transform the None values at axis 0 to [], so that they survive the next operation
    Reso = dak.firsts(Reso) #Chooses the first elements and flattens out, [] gets converted to None

    return Reso

In [31]:
#################################
#Begin the processor definition #
#################################
class mHrecoil(processor.ProcessorABC):
    '''
    mHrecoil example: e^+ + e^- rightarrow ZH rightarrow mu^+ mu^- + X(Recoil)
    Note: Use only BaseSchema with this processor
    '''
    def __init__(self, ecm):
        self.arg_ecm = ecm #\sqrt(s) in GeV
        self.arg_zmass = 91.0 #GeV
        
    def process(self,events):

        #Create a Packed Selection object to get a cutflow later
        cut = PackedSelection()
        cut.add('No cut', dak.ones_like(dak.num(get(events,'ReconstructedParticles','energy'),axis=1),dtype=bool))
        
        # Filter out any event with no reconstructed particles and generate Reconstructed Particle Attributes
        #ak.mask preserves array length
        at_least_one_recon = dak.num(get(events,'ReconstructedParticles','energy'), axis=1) > 0
        good_events = dak.mask(events,at_least_one_recon)
        cut.add('At least one Reco Particle', at_least_one_recon)
        
        Reco = get_all(good_events,'ReconstructedParticles')
        Muons = get_reco(Reco,'Muon#0',good_events)
        
        # Create Array of Muon Lorentz Vector
        Muon = ak.zip({"px":Muons.momentum_x,"py":Muons.momentum_y,"pz":Muons.momentum_z,"E":Muons.energy,"q":Muons.charge,}, with_name="Momentum4D")

        # Get Muons with a pt cut , if none of the muons in an event pass the cut, return none, ensuring the size of the cutflow
        pt_mask = dak.any(Muon.pt > 10, axis = 1)
        temp = dak.mask(Muon, pt_mask)
        Muon = Muon[temp.pt > 10]
        cut.add('At least one Muon pt > 10', pt_mask)

        # Get best Resonance around Z mass in an event
        Z_cand = Reso_builder(Muon, self.arg_zmass) 

        # Selection 0 : Z q=0 candidate
        q_mask = Z_cand.q == 0
        Z_cand_sel0 = dak.mask(Z_cand, q_mask)
        cut.add("Z_q = 0", q_mask)
        sel0_ocl = cut.cutflow(*cut.names).yieldhist()
        
        # Selection 1 : 80 < M_Z < 100
        Z_mass_mask = (Z_cand.mass > 80.0) & (Z_cand.mass < 100.0)
        Z_cand_sel1 = ak.mask(Z_cand, Z_mass_mask)
        cut.add('80 < $M_Z$ < 100',Z_mass_mask)
        sel = [*cut.names]
        sel.remove(sel[-2])
        sel1_ocl = cut.cutflow(*sel).yieldhist()
        
        #Recoil Calculation
        Recoil_sel0 = ak.zip({"px":0.0-Z_cand_sel0.px,"py":0.0-Z_cand_sel0.py,"pz":0.0-Z_cand_sel0.pz,"E":self.arg_ecm-Z_cand_sel0.E},with_name="Momentum4D")
        Recoil_sel1 = ak.zip({"px":0.0-Z_cand_sel1.px,"py":0.0-Z_cand_sel1.py,"pz":0.0-Z_cand_sel1.pz,"E":self.arg_ecm-Z_cand_sel1.E},with_name="Momentum4D")
        #Bug: Recoil computed this way are forced to have integer values, idk why
        
        #Prepare output
        #Choose the required histograms and their assigned variables to fill
        names = plot_props.columns.to_list()
        vars_sel0 = ([abs(Z_cand_sel0.mass)]*2) + ([abs(Recoil_sel0.mass)]*8)
        vars_sel1 = ([abs(Z_cand_sel1.mass)]*2) + ([abs(Recoil_sel1.mass)]*8)
        Output = {
            'histograms': {
                'sel0':{name:get_1Dhist(name,var) for name,var in zip(names,vars_sel0)},
                'sel1':{name:get_1Dhist(name,var) for name,var in zip(names,vars_sel1)}
            },
            'cutflow': {
                'sel0': {'Onecut':sel0_ocl[0],'Cutflow':sel0_ocl[1],'Labels':sel0_ocl[2]},
                'sel1': {'Onecut':sel1_ocl[0],'Cutflow':sel1_ocl[1],'Labels':sel1_ocl[2]}
            }
        }
        return Output

    def postprocess(self, accumulator):
        pass


In [32]:
from coffea.nanoevents import NanoEventsFactory, BaseSchema
import dask
events,report = NanoEventsFactory.from_root(
    '../data/p8_ee_ZH_ecm240/events_082532938.root:events',
    schemaclass=BaseSchema,
    metadata={'dataset':'ZH'},
    uproot_options={"allow_read_errors_with_report": True}
).events()

In [33]:
p = mHrecoil(ecm=240.0)
out = p.process(events)
(computed,) = dask.compute(out)

In [34]:
computed

{'histograms': {'sel0': {'Zm': Hist(Regular(100, 0, 250, label='Axis 0'), storage=Double()) # Sum: 2870.0,
   'Zm_zoom': Hist(Regular(40, 80, 100, label='Axis 0'), storage=Double()) # Sum: 2381.0 (2870.0 with flow),
   'Recoilm': Hist(Regular(100, 0, 200, label='Axis 0'), storage=Double()) # Sum: 2846.0 (2870.0 with flow),
   'Recoilm_zoom': Hist(Regular(200, 80, 160, label='Axis 0'), storage=Double()) # Sum: 2675.0 (2870.0 with flow),
   'Recoilm_zoom1': Hist(Regular(100, 120, 140, label='Axis 0'), storage=Double()) # Sum: 2399.0 (2870.0 with flow),
   'Recoilm_zoom2': Hist(Regular(200, 120, 140, label='Axis 0'), storage=Double()) # Sum: 2399.0 (2870.0 with flow),
   'Recoilm_zoom3': Hist(Regular(400, 120, 140, label='Axis 0'), storage=Double()) # Sum: 2399.0 (2870.0 with flow),
   'Recoilm_zoom4': Hist(Regular(800, 120, 140, label='Axis 0'), storage=Double()) # Sum: 2399.0 (2870.0 with flow),
   'Recoilm_zoom5': Hist(Regular(2000, 120, 140, label='Axis 0'), storage=Double()) # Sum: 2

In [7]:
Reco = get_all(events, "ReconstructedParticles")
Muons = get_reco(Reco,'Muon#0',events)
Muon = ak.zip({"px":Muons.momentum_x,"py":Muons.momentum_y,"pz":Muons.momentum_z,"E":Muons.energy,"q":Muons.charge,}, with_name="Momentum4D")

In [8]:
((Muon.px**2 + Muon.py**2)**0.5).compute()

<Array [[], [], [], [61.1], ..., [], [], [], []] type='100000 * var * float32'>

In [9]:
Muon.pt.compute()

<Array [[], [], [], [61.1], ..., [], [], [], []] type='100000 * var * float32'>

In [11]:
#Create all the combinations
combs = dak.combinations(Muon,2)

# Get diMuons
Muon1 , Muon2 = dak.unzip(combs)
di_Muon = Muon1 + Muon2

# An event should have at least one dimuon pair composed of muons with opposite charges
q_sum_mask = dak.any((Muon1.q + Muon2.q) == 0, axis=1)
# An event should have at least one dimuon pair composed of muons with pt>10
pt_mask = dak.any(Muon1.pt > 10, axis=1) & dak.any(Muon2.pt > 10, axis=1) #this means at least two muons with pt > 10

di_Muon = dak.mask(di_Muon , q_sum_mask)
di_Muon = dak.mask(di_Muon , pt_mask)

# Sort by closest mass to the resonance value
sort_mask = dak.argsort(abs(91.0-di_Muon.mass), axis=1)
Reso = di_Muon[sort_mask]

# Choose events with exactly one Z

In [12]:
def clean(arr):
    '''
    Remove none values from axis 0
    '''
    temp = dak.fill_none(arr,[],axis=0)
    return temp[dak.num(temp, axis=1) > 0]

In [13]:
clean(Reso).mass.compute()

<Array [[85.4], [88.8], ..., [87.4], [89.4]] type='2883 * var * float32'>

In [14]:
ak.firsts(clean(Reso)).mass.compute()

<Array [85.4, 88.8, 86.6, 92.3, ..., 42.2, 87.4, 89.4] type='2883 * ?float32'>

In [15]:
one_Z = dak.num(Reso,axis=1) == 1
Reson = dak.mask(Reso, one_Z)
clean(Reson).mass.compute()

<Array [[85.4], [88.8], ..., [87.4], [89.4]] type='2688 * var * float32'>

In [16]:
# Difference in a smaller dataset both in coffea
(2883-2688)*100/2688

7.254464285714286

In [17]:
# Difference in the bigger dataset between coffea and fcc-analyses
(58793-54696)*100/54696

7.4904929062454295

In [18]:
Reso_ = dak.fill_none(Reso,[],axis=0)
r = dak.firsts(Reso_).mass

In [19]:
~dak.is_none(r, axis=0).compute()

<Array [False, False, False, ..., False, False, False] type='100000 * bool'>

In [20]:
Muon

dask.awkward<zip, npartitions=1>

In [25]:
# Get Muons with a cut , if none of them pass it in an event, return none
temp = dak.mask(Muon, dak.any(Muon.pt > 10, axis = 1))
Muon_ = Muon[temp.pt > 10]

In [26]:
Muon_.pt.compute()

<Array [None, None, None, ..., None, None] type='100000 * option[var * floa...'>

In [27]:
Muon_[dak.num(Muon) > 1].pt.compute()

<Array [[56.6, 19.4], [...], ..., [53.4, 13]] type='3352 * option[var * flo...'>

In [20]:
sel = ['a','b','c']
sel.remove('c')

In [22]:
sel[-1]

'b'